In [ ]:
use synthetic dataset 
P9 (OTS 2) : Design and implement a neural network with a given specification for the churn modelling dataset. 
Show train and test accuracy, and confusion matrix.
• Split the dataset into a train set and a test set. Test size = 33%
• Feature scaling (preprocessing using standard scaler)
• Model: Input layer, hidden layer 1 (6 units, ReLu), hidden layer 2 (6 units ReLu), output layer (one unit, sigmoid).
• Loss function: Binary cross entropy.
• Mini-batch, batch_size = 10, epoch = 100.
• Optimizer = RMSprop
• In case of overfitting apply a dropout layer

Load the "Churn_Modelling.csv" dataset
Split the dataset into train (67%) and test (33%) sets
Apply feature scaling using StandardScaler
Implement a neural network with:

Input layer
Hidden layer 1 (6 units, ReLU)
Hidden layer 2 (6 units, ReLU)
Output layer (1 unit, sigmoid)


Use binary cross-entropy loss function
Use mini-batch training with batch_size=10 and 100 epochs
Use RMSprop optimizer
Add dropout if we detect overfitting
Evaluate with train/test accuracy and confusion matrix

In [2]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import standardscaler 




ImportError: cannot import name 'standardscaler' from 'sklearn.preprocessing' (C:\Users\FAIZ SIDDIQUI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\preprocessing\__init__.py)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# 1. Load the dataset
print("Loading the dataset...")
dataset = pd.read_csv('Churn_Modelling.csv')
print(f"Dataset shape: {dataset.shape}")
print(dataset.head())

# 2. Preprocessing
print("\nPreprocessing the data...")
# Drop unnecessary columns (like ID, Name, etc.)
X = dataset.iloc[:, 3:-1].values  # Skip customer ID, surname, row number
y = dataset.iloc[:, -1].values

# Encode categorical variables
# Assuming there are categorical variables like gender, geography, etc.
X_encoded = pd.get_dummies(pd.DataFrame(X), drop_first=True).values

# 3. Split dataset into train and test sets (67% train, 33% test)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.33, random_state=42)
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# 4. Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5. Build the neural network model
input_dim = X_train.shape[1]  # Number of features

print("\nBuilding the neural network model...")
model = Sequential([
    # Input layer and first hidden layer with 6 units and ReLU activation
    Dense(6, activation='relu', input_shape=(input_dim,)),
    
    # Second hidden layer with 6 units and ReLU activation
    Dense(6, activation='relu'),
    
    # Output layer with sigmoid activation for binary classification
    Dense(1, activation='sigmoid')
])

# Compile the model with binary cross-entropy loss and RMSprop optimizer
model.compile(
    optimizer=RMSprop(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Model summary
model.summary()

# 6. Train the model
print("\nTraining the model...")
history = model.fit(
    X_train, y_train,
    batch_size=10,
    epochs=100,
    validation_data=(X_test, y_test),
    verbose=1
)

# 7. Evaluate the model
print("\nEvaluating the model...")
# Training accuracy
y_train_pred = (model.predict(X_train) > 0.5).astype("int32")
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")

# Test accuracy
y_test_pred = (model.predict(X_test) > 0.5).astype("int32")
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

# 8. Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)
print("\nConfusion Matrix:")
print(cm)

# 9. Check for overfitting
print("\nChecking for overfitting...")
if train_accuracy - test_accuracy > 0.05:
    print("Model shows signs of overfitting. Implementing model with dropout layers...")
    
    # Rebuild the model with dropout layers
    model_with_dropout = Sequential([
        # Input layer and first hidden layer
        Dense(6, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),  # Add dropout after first hidden layer
        
        # Second hidden layer
        Dense(6, activation='relu'),
        Dropout(0.2),  # Add dropout after second hidden layer
        
        # Output layer
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    model_with_dropout.compile(
        optimizer=RMSprop(),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Train the model with dropout
    history_dropout = model_with_dropout.fit(
        X_train, y_train,
        batch_size=10,
        epochs=100,
        validation_data=(X_test, y_test),
        verbose=1
    )
    
    # Evaluate the model with dropout
    y_train_pred_dropout = (model_with_dropout.predict(X_train) > 0.5).astype("int32")
    train_accuracy_dropout = accuracy_score(y_train, y_train_pred_dropout)
    
    y_test_pred_dropout = (model_with_dropout.predict(X_test) > 0.5).astype("int32")
    test_accuracy_dropout = accuracy_score(y_test, y_test_pred_dropout)
    
    print(f"Training Accuracy with Dropout: {train_accuracy_dropout:.4f}")
    print(f"Test Accuracy with Dropout: {test_accuracy_dropout:.4f}")
    
    # Confusion Matrix for model with dropout
    cm_dropout = confusion_matrix(y_test, y_test_pred_dropout)
    print("\nConfusion Matrix with Dropout:")
    print(cm_dropout)
    
    # Visualize training history
    plt.figure(figsize=(12, 5))
    
    # Plot training history for model without dropout
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='train accuracy')
    plt.plot(history.history['val_accuracy'], label='test accuracy')
    plt.title('Model without Dropout')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot training history for model with dropout
    plt.subplot(1, 2, 2)
    plt.plot(history_dropout.history['accuracy'], label='train accuracy')
    plt.plot(history_dropout.history['val_accuracy'], label='test accuracy')
    plt.title('Model with Dropout')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No significant overfitting detected.")
    
    # Visualize training history
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='train accuracy')
    plt.plot(history.history['val_accuracy'], label='test accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='train loss')
    plt.plot(history.history['val_loss'], label='test loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

print("\nNeural network implementation complete!")

ModuleNotFoundError: No module named 'tensorflow'

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Create a synthetic churn dataset
def generate_synthetic_churn_data(n_samples=1000):
    # Features
    customer_id = np.arange(1, n_samples+1)
    credit_score = np.random.randint(300, 900, n_samples)
    age = np.random.randint(18, 95, n_samples)
    tenure = np.random.randint(0, 11, n_samples)
    balance = np.random.normal(50000, 30000, n_samples)
    num_of_products = np.random.randint(1, 5, n_samples)
    has_credit_card = np.random.randint(0, 2, n_samples)
    is_active_member = np.random.randint(0, 2, n_samples)
    estimated_salary = np.random.normal(70000, 30000, n_samples)
    
    # Calculate churn probability based on features
    churn_prob = 1 / (1 + np.exp(-(
        -0.5 
        - 0.003 * (credit_score - 650) 
        + 0.02 * (age - 40) 
        - 0.1 * tenure 
        + 0.000001 * balance 
        + 0.2 * (num_of_products - 1) 
        - 0.3 * has_credit_card 
        - 0.5 * is_active_member 
        - 0.000002 * estimated_salary
    )))
    
    # Generate actual churn values
    exited = (np.random.random(n_samples) < churn_prob).astype(int)
    
    # Create DataFrame
    data = {
        'CustomerId': customer_id,
        'CreditScore': credit_score,
        'Age': age,
        'Tenure': tenure,
        'Balance': balance,
        'NumOfProducts': num_of_products,
        'HasCrCard': has_credit_card,
        'IsActiveMember': is_active_member,
        'EstimatedSalary': estimated_salary,
        'Exited': exited
    }
    
    return pd.DataFrame(data)

In [16]:

df = generate_synthetic_churn_data(10000)
print("Synthetic Churn Dataset Sample:")
print(df.head())
print("\nChurn Distribution:")
print(df['Exited'].value_counts(normalize=True))

X = df.drop(['CustomerId', 'Exited'], axis=1).values
y = df['Exited'].values.reshape(-1, 1)

# Split the dataset (Test size = 33%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Feature scaling using standard scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def sigmoid(x):
    x = np.clip(x, -500, 500)  # Prevent overflow
    return 1 / (1 + np.exp(-x))

Synthetic Churn Dataset Sample:
   CustomerId  CreditScore  Age  ...  IsActiveMember  EstimatedSalary  Exited
0           1          402   18  ...               1     78267.702848       0
1           2          735   77  ...               0     58847.439468       1
2           3          570   74  ...               1     68714.631536       0
3           4          406   20  ...               0     53216.964630       0
4           5          371   35  ...               0     59573.769839       0

[5 rows x 10 columns]

Churn Distribution:
Exited
0    0.6465
1    0.3535
Name: proportion, dtype: float64


In [17]:
def create_mini_batches(X, y, batch_size):
    mini_batches = []
    data = np.hstack((X, y))
    np.random.shuffle(data)
    n_minibatches = data.shape[0] // batch_size
    
    for i in range(n_minibatches):
        mini_batch = data[i * batch_size:(i + 1) * batch_size, :]
        X_mini = mini_batch[:, :-1]
        y_mini = mini_batch[:, -1:] 
        mini_batches.append((X_mini, y_mini))
    
    
    if data.shape[0] % batch_size != 0:
        mini_batch = data[n_minibatches * batch_size:, :]
        X_mini = mini_batch[:, :-1]
        y_mini = mini_batch[:, -1:]
        mini_batches.append((X_mini, y_mini))
    
    return mini_batches

In [19]:


class NeuralNetwork:
    def __init__(self, input_size, use_dropout=False, dropout_rate=0.2):
        self.input_size = input_size
        self.use_dropout = use_dropout
        self.dropout_rate = dropout_rate
        
      
        self.W1 = np.random.randn(input_size, 6) * np.sqrt(2. / input_size)
        self.b1 = np.zeros((1, 6))
        
        self.W2 = np.random.randn(6, 6) * np.sqrt(2. / 6)
        self.b2 = np.zeros((1, 6))
        
        self.W3 = np.random.randn(6, 1) * np.sqrt(2. / 6)
        self.b3 = np.zeros((1, 1))
        
        
        self.learning_rate = 0.001
        self.beta = 0.9  # Decay rate for moving average
        
        p
        self.v_W1 = np.zeros_like(self.W1)
        self.v_b1 = np.zeros_like(self.b1)
        self.v_W2 = np.zeros_like(self.W2)
        self.v_b2 = np.zeros_like(self.b2)
        self.v_W3 = np.zeros_like(self.W3)
        self.v_b3 = np.zeros_like(self.b3)
        
        self.epsilon = 1e-8  # Small value to avoid division by zero
    
    def forward(self, X, training=True):
        # Input to first hidden layer
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = relu(self.Z1)
        
    
        if training and self.use_dropout:
            self.D1 = np.random.rand(*self.A1.shape) > self.dropout_rate
            self.A1 *= self.D1
            
            self.A1 /= (1 - self.dropout_rate)
        
        
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = relu(self.Z2)
        
        if training and self.use_dropout:
            self.D2 = np.random.rand(*self.A2.shape) > self.dropout_rate
            self.A2 *= self.D2
            # Scale the values
            self.A2 /= (1 - self.dropout_rate)
        
        
        self.Z3 = np.dot(self.A2, self.W3) + self.b3
        self.A3 = sigmoid(self.Z3)
        
        return self.A3
    
    def compute_loss(self, y_true, y_pred):

        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return loss
    
    def backward(self, X, y):
        m = X.shape[0]
        
        
        dZ3 = self.A3 - y
        dW3 = (1/m) * np.dot(self.A2.T, dZ3)
        db3 = (1/m) * np.sum(dZ3, axis=0, keepdims=True)
        
        
        dA2 = np.dot(dZ3, self.W3.T)
        if self.use_dropout:
            dA2 *= self.D2
            dA2 /= (1 - self.dropout_rate)
        dZ2 = dA2 * relu_derivative(self.Z2)
        dW2 = (1/m) * np.dot(self.A1.T, dZ2)
        db2 = (1/m) * np.sum(dZ2, axis=0, keepdims=True)
        
        